#### Hypothesis: Repos with shorter descriptions are more likely to be dead than repos with long descriptions. A short description, I believe, signifies a repo that is used for a simple backup, or is a project started by an individual who got in over their head or who otherwise could not follow through with completion of the project.

#### I am defining a "dead" repo as one that has not been updated for over a year. I believe that, the shorter the description, the greater the liklihood that the repo is dead, with repos with a single subscribers being most likely to be dead or completely unused.

In [ ]:
import pymongo, re, datetime

SECONDS_IN_YEAR = 3.154e7

client = pymongo.MongoClient( 'da0.eecs.utk.edu' )
db = client['github']
readcoll = db['repos']

date = ''
fork = ''

rec = 0

for r in readcoll.find( {} ):
    if r is None or 'values' not in r:
        continue
    rec += 1
    
print( 'number of records: ' + str( rec ) )

In [ ]:
import pymongo, re, datetime

SECONDS_IN_YEAR = 3.154e7

client = pymongo.MongoClient( 'da0.eecs.utk.edu' )
db = client['github']
readcoll = db['repos']

date = ''
fork = ''

gho = open( 'jteague6_github', 'w' )

for r in readcoll.find( {} ):
    if r is None or 'values' not in r:
        continue
    for v in r['values']:
        for vv in v.keys():
            #print( vv )
            #print ('values.' + vv + ':' + str(v[vv]))
            if( vv == 'description' ):
                if v[vv] is None:
                    desclen = -1
                else:
                    desclen =  str( len( v[vv] ) )
            elif ( vv == 'updated_at' ):
                if v[vv] is None:
                    dto = datetime.datetime.now()
                else:
                    date = str( v[vv] )
                    idx  = date.find( 'T' )
                #print( str( idx ) + " " + date[:idx] )
                    dto = datetime.datetime.strptime(date[:idx], "%Y-%m-%d")
                #print( dto )
                #print( ( datetime.datetime.now() - dto ).total_seconds() )
            
    if desclen == -1:
        continue
        
    gho.write( str( desclen ) + " " + str( ( datetime.datetime.now() - dto ).total_seconds() ) + '\n' )
    
gho.close()
print( 'done' )

In [47]:
import pymongo, re, datetime

SECONDS_IN_YEAR = 3.154e7

client = pymongo.MongoClient( 'da0.eecs.utk.edu' )
db = client['bitbucket']
readcoll = db['repos']

date = ''
fork = ''

bbo = open( 'jteague6_bitbucket', 'w' )

for r in readcoll.find( {} ):
    if r is None:
        continue
    for v in r.keys():
        updated_on = datetime.datetime.now()
        idx = -1
        if v == 'description' and r[v]:
            #print( 'description: ' + str( r[v] ) )
            desclen = len( r[v] )
            #print( 'desclen: ' + str( desclen ) )
        elif v == 'updated_on' and r[v]:
            #print( 'updated_on: ' + str( r[v] ) )
            idx = r[v].find( 'T' )
            updated_on = datetime.datetime.strptime(r[v][:idx], "%Y-%m-%d")
            #print( updated_on )
            dto = ( datetime.datetime.now() - updated_on ).total_seconds()
    bbo.write( str( desclen ) + " " + '{:10.6f}'.format( dto ) + '\n' )
    
bbo.close()
print( 'done' )

done


Exception ignored in: <bound method Cursor.__del__ of <pymongo.cursor.Cursor object at 0x7f002e899e80>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.4/dist-packages/pymongo/cursor.py", line 211, in __del__
    self.__die()
  File "/usr/local/lib/python3.4/dist-packages/pymongo/cursor.py", line 271, in __die
    self.__address)
  File "/usr/local/lib/python3.4/dist-packages/pymongo/mongo_client.py", line 833, in close_cursor
    self.__cursor_manager.close(cursor_id, address)
  File "/usr/local/lib/python3.4/dist-packages/pymongo/cursor_manager.py", line 56, in close
    self.__client().kill_cursors([cursor_id], address)
AttributeError: 'NoneType' object has no attribute 'kill_cursors'


Okay, so that didn't work too well. These files are huge, and R takes forever to process the information (I should have asked if there's a way to run the notebook in the background).

Here I'm going to truncate each file to 25,000 records and replace any values of -1 with 0. 25,000 is still a lot, and since they're apparently not in any particular order I'm really hoping I still get good results.

Also, while I'm at it, I'll convert the number of seconds to years.

In [12]:
SECONDS_IN_YEAR = 3.154e7

gho = open( 'jteague6_bitbucket', 'r' )
ghot = open( 'jteague6_bitbucket_truncated', 'w' )

i = 0
for l in gho.readlines():
    if i > 25000:
        break
        
    ls = l.split()
    desc = ls[0]
    time = ls[1]
    
    if desc == -1:
        desc = 0
        
    time = float(time) / SECONDS_IN_YEAR
    
    ghot.write( str( desc ) + " " + '{:10.6f}'.format( time ) + '\n' )
    i = i + 1

In [15]:
SECONDS_IN_YEAR = 3.154e7

gho = open( 'jteague6_github', 'r' )
ghot = open( 'jteague6_github_truncated', 'w' )

i = 0
for l in gho.readlines():
    if i > 25000:
        break
        
    ls = l.split()
    desc = int( ls[0] )
    time = ls[1]
    
    if desc == -1:
        continue
    
    # I hate doing this, but 0 screws up the log calculations, so all zeroes have to be changed to ones
    if desc == 0:
        desc = 1;
        
    time = float(time) / SECONDS_IN_YEAR
    
    ghot.write( str( desc ) + " " + '{:10.6f}'.format( time ) + '\n' )
    i = i + 1